In [1]:
import pandas as pd 
import numpy as np

In [2]:
df_air =pd.read_csv("D:/Airtel Prepaid.csv")
df_jio = pd.read_csv("D:/Jio - Jio Prepaid Planes.csv")

In [3]:
df_air.head()

,Price,Data_per_Day,Data,Days,Additional_Benefits
0,3359,2.5,0,365,Disney + Hotstar 1 year
1,2999,2.0,0,365,0
2,999,2.5,0,84,Amazon prime membership
3,839,2.0,0,84,Disney + Hotstar 3 Months
4,719,1.5,0,84,0


In [4]:
df_jio.head()

,Days,Price,Price_Day,For_365_Days,Profit_per_customer,From_400_Million_users,Data_per_day
0,28,209,7.464286,2724.464286,5.214286,2085.714286,1.0
1,24,179,7.458333,2722.291667,3.041667,1216.666667,1.0
2,20,149,7.450000,2719.250000,0.000000,0.000000,1.0
3,336,2545,7.574405,2764.657738,0.000000,0.000000,1.5
4,84,666,7.928571,2893.928571,129.270833,51708.333330,1.5


In [5]:
df_jio.dtypes

Days                        int64
Price                       int64
Price_Day                 float64
For_365_Days              float64
Profit_per_customer       float64
From_400_Million_users    float64
Data_per_day              float64
dtype: object

In [6]:
df_air.dtypes

Price                    int64
Data_per_Day           float64
Data                     int64
Days                     int64
Additional_Benefits     object
dtype: object

In [7]:
df_air.isnull().sum()

Price                  0
Data_per_Day           0
Data                   0
Days                   0
Additional_Benefits    0
dtype: int64

In [8]:
df_jio.isnull().sum()

Days                      0
Price                     0
Price_Day                 0
For_365_Days              0
Profit_per_customer       0
From_400_Million_users    0
Data_per_day              0
dtype: int64

In [9]:
df_jio.duplicated().value_counts()

False    16
Name: count, dtype: int64

In [10]:
df_air.duplicated().value_counts()

False    23
True      5
Name: count, dtype: int64

In [11]:


# Rename columns for easier access (SQL-like aliases)
df_jio = df_jio.rename(columns={
    'Data_per_day': 'Data_Day', 
    'Profit_per_customer': 'Profit',
    'Price_Day': 'Price_Day_Calc' # Renaming existing calc column
})

df_air = df_air.rename(columns={
    'Data_per_Day': 'Data_Day', 
    'Data': 'Total_Data', 
    'Additional_Benefits': 'Benefits'
})

print("Data Loaded Successfully!")

jio = df_jio
air = df_air

Data Loaded Successfully!


#### 1. Retrieve all High-Speed Jio Plans (> 2GB/Day

In [12]:
Q1 = jio[jio['Data_Day'] > 2]
print(Q1[['Days', 'Price', 'Data_Day']])

    Days  Price  Data_Day
14    84   1199       3.0
15    28    419       3.0


#### 2. Find Long-Term Airtel Plans (84 Days or more)

In [13]:
Q2 = air[air['Days']>= 84]
print(Q2[['Price', 'Days', 'Data_Day']])

    Price  Days  Data_Day
0    3359   365       2.5
1    2999   365       2.0
2     999    84       2.5
3     839    84       2.0
4     719    84       1.5
11    455    84       0.0
23    839    84       2.0
26   3359   365       2.5
27   1799   365       0.0


 #### 3. Identify Budget Friendly Plans (< ₹300) for both

In [14]:
Jio_Budget = jio[jio['Price'] < 300 ][['Price', 'Days']].copy()
Jio_Budget['Provider'] = 'Jio'

Airtel_Budget = air[air['Price'] < 300][['Price', 'Days']].copy()
Airtel_Budget['Provider'] = 'Airtel'

Q3 = pd.concat([Jio_Budget,Airtel_Budget])
Q3

,Price,Days,Provider
0,209,28,Jio
1,179,24,Jio
2,149,20,Jio
6,259,30,Jio
7,239,28,Jio
8,199,23,Jio
9,119,14,Jio
12,299,28,Jio
13,249,23,Jio
15,299,28,Airtel


#### 4. Find "Data-Only" Plans in Airtel

In [15]:
Q4 = air[(air['Data_Day'] == 0) & (air['Total_Data'] > 0)]
print(Q4[['Price', 'Total_Data', 'Days']]) 

    Price  Total_Data  Days
11    455           6    84
16    296          25    30
27   1799          24   365


#### 5. List all plans with exactly 28 days validity

In [16]:
Q5 = jio[jio['Days'] == 28].copy()
Q5['Provider'] = 'JIO'
print(Q5[['Price', 'Days','Data_Day','Provider']])

    Price  Days  Data_Day Provider
0     209    28       1.0      JIO
7     239    28       1.5      JIO
12    299    28       2.0      JIO
15    419    28       3.0      JIO


#### 6. Find Airtel Plans with "Disney+ Hotstar"

In [17]:
Q6 = air[air['Benefits'].str.contains('Disney', case= False, na=False)].copy()
Q6['Provider'] = 'Airtel'
print(Q6[['Price','Days','Benefits','Provider']])

    Price  Days                   Benefits Provider
0    3359   365    Disney + Hotstar 1 year   Airtel
3     839    84  Disney + Hotstar 3 Months   Airtel
7     599    28    Disney + Hotstar 1 year   Airtel
9     499    28    Disney + Hotstar 1 year   Airtel
12    399    28  Disney + Hotstar 3 Months   Airtel
22    399    28  Disney + Hotstar 3 Months   Airtel
23    839    84  Disney + Hotstar 3 Months   Airtel
24    499    28    Disney + Hotstar 1 year   Airtel
25    599    28    Disney + Hotstar 1 year   Airtel
26   3359   365    Disney + Hotstar 1 year   Airtel


#### 7. Find Airtel Plans with "Amazon Prime"

In [18]:
Q7 = air[air['Benefits'].str.contains('Amazon', case= False, na=False)].copy()
Q7['Provider'] = 'Jio'
print(Q6[['Price','Days','Benefits','Provider']])

    Price  Days                   Benefits Provider
0    3359   365    Disney + Hotstar 1 year   Airtel
3     839    84  Disney + Hotstar 3 Months   Airtel
7     599    28    Disney + Hotstar 1 year   Airtel
9     499    28    Disney + Hotstar 1 year   Airtel
12    399    28  Disney + Hotstar 3 Months   Airtel
22    399    28  Disney + Hotstar 3 Months   Airtel
23    839    84  Disney + Hotstar 3 Months   Airtel
24    499    28    Disney + Hotstar 1 year   Airtel
25    599    28    Disney + Hotstar 1 year   Airtel
26   3359   365    Disney + Hotstar 1 year   Airtel


#### 8. Identify Plans with NO Additional Benefits

In [19]:
Q8 = air[(air['Benefits'] == '0')  | (air['Benefits'].isna())]
print(Q8[['Price', 'Days','Data_Day','Benefits']])

    Price  Days  Data_Day Benefits
1    2999   365       2.0        0
4     719    84       1.5        0
6     666    77       1.5        0
8     549    56       2.0        0
10    479    56       1.5        0
11    455    84       0.0        0
14    319    30       2.0        0
15    299    28       1.5        0
16    296    30       0.0        0
27   1799   365       0.0        0


#### 9. Calculate "Cost Per GB" for Jio

In [20]:
Q9 = jio[jio['Data_Day'] >  0].copy()
Q9['Cost_Per_GB'] = Q9['Price'] / (Q9['Data_Day'] * Q9['Days'])
Q9 =Q9.sort_values(by='Cost_Per_GB')
print(Q9[['Price', 'Days', 'Data_Day', 'Cost_Per_GB' ]].head())

    Price  Days  Data_Day  Cost_Per_GB
10   2879   365       2.0     3.943836
14   1199    84       3.0     4.757937
11    533    56       2.0     4.758929
15    419    28       3.0     4.988095
3    2545   336       1.5     5.049603


#### 10. Calculate "True Monthly Cost" (Normalized to 30 days)

In [ ]:
# AIRTEL

air['Est_Monthly_Cost'] = (air['Price'] / air['Days']) * 30
print(air[['Price', 'Days','Est_Monthly_Cost' ]].head())

   Price  Days  Est_Monthly_Cost
0   3359   365        276.082192
1   2999   365        246.493151
2    999    84        356.785714
3    839    84        299.642857
4    719    84        256.785714


In [ ]:
# JIO

jio['Est_Monthly_Cost'] = (jio['Price'] / jio['Days']) * 30
print(jio[['Price', 'Days','Est_Monthly_Cost' ]].head())

    Price  Days  Est_Monthly_Cost
10   2879   365        236.630137
3    2545   336        227.232143
14   1199    84        428.214286
4     666    84        237.857143
11    533    56        285.535714


#### 11. Categorize Jio Plans (Budget vs Premium)

In [22]:
def categorize(Price):
    if Price < 200: return 'Budget'
    elif 200 <= Price <= 600: return 'Standard'
    else : return 'Premium'

jio['Plan_Category'] = jio['Price'].apply(categorize)
jio = jio.sort_values(by='Price',ascending=False)
print(jio[['Price','Data_Day','Plan_Category']].head(15))

    

    Price  Data_Day Plan_Category
10   2879       2.0       Premium
3    2545       1.5       Premium
14   1199       3.0       Premium
4     666       1.5       Premium
11    533       2.0      Standard
5     479       1.5      Standard
15    419       3.0      Standard
12    299       2.0      Standard
6     259       1.5      Standard
13    249       2.0      Standard
7     239       1.5      Standard
0     209       1.0      Standard
8     199       1.5        Budget
1     179       1.0        Budget
2     149       1.0        Budget


#### 12. Estimate Yearly Cost

In [ ]:
# JIO

jio['Yearly_Projection'] = (365 / jio['Days']) * jio['Price']
print(jio[['Price', 'Days', 'Yearly_Projection']])

    Price  Days  Yearly_Projection
10   2879   365        2879.000000
3    2545   336        2764.657738
14   1199    84        5209.940476
4     666    84        2893.928571
11    533    56        3474.017857
5     479    56        3122.053571
15    419    28        5461.964286
12    299    28        3897.678571
6     259    30        3151.166667
13    249    23        3951.521739
7     239    28        3115.535714
0     209    28        2724.464286
8     199    23        3158.043478
1     179    24        2722.291667
2     149    20        2719.250000
9     119    14        3102.500000


In [ ]:
# AIRTEL

air['Yearly_Projection'] = (365 / air['Days']) * air['Price']
print(air[['Price', 'Days', 'Yearly_Projection']])

    Price  Days  Yearly_Projection
0    3359   365        3359.000000
1    2999   365        2999.000000
2     999    84        4340.892857
3     839    84        3645.654762
4     719    84        3124.226190
5     699    56        4555.982143
6     666    77        3157.012987
7     599    28        7808.392857
8     549    56        3578.303571
9     499    28        6504.821429
10    479    56        3122.053571
11    455    84        1977.083333
12    399    28        5201.250000
13    359    28        4679.821429
14    319    30        3881.166667
15    299    28        3897.678571
16    296    30        3601.333333
17    265    28        3454.464286
18    239    24        3634.791667
19    209    21        3632.619048
20    179    28        2333.392857
21    155    24        2357.291667
22    399    28        5201.250000
23    839    84        3645.654762
24    499    28        6504.821429
25    599    28        7808.392857
26   3359   365        3359.000000
27   1799   365     

#### 14. Total Network Data Capacity (Jio)

In [24]:
Total_capacity = (jio['Data_Day'] *  jio['Days']).sum()
print(f"Total_capacity : {Total_capacity} GB")

Total_capacity : 2208.5 GB


#### 15. Count of Plans per Data Band

In [ ]:
# AIRTEL

Q15 = air['Data_Day'].value_counts().sort_index()
print(Q15)

Data_Day
0.0    3
1.0    4
1.5    4
2.0    9
2.5    5
3.0    3
Name: count, dtype: int64


In [ ]:
# JIO

Q15 = jio['Data_Day'].value_counts().sort_index()
print(Q15)

Data_Day
1.0    3
1.5    7
2.0    4
3.0    2
Name: count, dtype: int64


#### 16. Min/Max/Avg Price Analysis

In [ ]:
# AIRTEL

stats = air['Price'].agg(['min','max','mean'])
print(stats)

min      155.000000
max     3359.000000
mean     824.071429
Name: Price, dtype: float64


In [ ]:
# JIO

stats = jio['Price'].agg(['min','max','mean'])
print(stats)

min      119.0000
max     2879.0000
mean     663.8125
Name: Price, dtype: float64


#### 17. The "Best Value" Leaderboard

In [38]:
# Prepare Jio Data

jio_clean = jio[['Price', 'Days','Data_Day']].copy()
jio_clean['Provider'] = 'Jio'
jio_clean['Daily_Cost'] = jio_clean['Price'] /jio_clean['Days']


# Prepare Airtel Data

airtel_clean = air[['Price', 'Days','Data_Day']].copy()
airtel_clean['Provider'] = 'Airtel'
airtel_clean['Daily_Cost'] = airtel_clean['Price'] / airtel_clean['Days']

# Combine and Sort

leaderboard = pd.concat([jio_clean,airtel_clean]).sort_values(by='Daily_Cost').head(10)
print(leaderboard)

    Price  Days  Data_Day Provider  Daily_Cost
27   1799   365       0.0   Airtel    4.928767
11    455    84       0.0   Airtel    5.416667
20    179    28       2.0   Airtel    6.392857
21    155    24       1.0   Airtel    6.458333
2     149    20       1.0      Jio    7.450000
1     179    24       1.0      Jio    7.458333
0     209    28       1.0      Jio    7.464286
3    2545   336       1.5      Jio    7.574405
10   2879   365       2.0      Jio    7.887671
4     666    84       1.5      Jio    7.928571


#### 18. Compare 84-Day Plans Head-to-Head (Join)

In [63]:
# Merge only plans with 84 days

jio_84 = jio[jio['Days'] == 84]
airtel_84 =air[air['Days'] == 84]


comaprison = pd.merge(jio_84,airtel_84, on='Days', suffixes=('_jio', '_airtel'))
comaprison['Price_Diff'] = comaprison['Price_airtel'] - comaprison['Price_jio']


cols = ['Days', 'Price_jio', 'Price_airtel', 'Data_Day_jio', 'Data_Day_airtel', 'Price_Diff']

print(comaprison[cols])

   Days  Price_jio  Price_airtel  Data_Day_jio  Data_Day_airtel  Price_Diff
0    84       1199           999           3.0              2.5        -200
1    84       1199           839           3.0              2.0        -360
2    84       1199           719           3.0              1.5        -480
3    84       1199           455           3.0              0.0        -744
4    84       1199           839           3.0              2.0        -360
5    84        666           999           1.5              2.5         333
6    84        666           839           1.5              2.0         173
7    84        666           719           1.5              1.5          53
8    84        666           455           1.5              0.0        -211
9    84        666           839           1.5              2.0         173


#### 19. Rank Plans by Data Value

In [ ]:
# Airtel

air['Data_Rank'] = air['Data_Day'].rank(ascending=False, method='min')
print(air[['Price', 'Data_Day', 'Data_Rank']].sort_values('Data_Rank').head())

    Price  Data_Day  Data_Rank
7     599       3.0        1.0
5     699       3.0        1.0
25    599       3.0        1.0
0    3359       2.5        4.0
12    399       2.5        4.0


In [70]:
# Jio

jio['Data_Rank'] = jio['Data_Day'].rank(ascending=False, method='min')
print ('     Data Value by Rank')
print(jio[['Price', 'Data_Day', 'Data_Rank']].sort_values('Data_Rank').head())

     Data Value by Rank
    Price  Data_Day  Data_Rank
14   1199       3.0        1.0
15    419       3.0        1.0
11    533       2.0        3.0
10   2879       2.0        3.0
12    299       2.0        3.0


#### 20. Jio Profit Drivers

In [71]:
jio['Margin_percentage'] = (jio['Profit'] / jio['Price']) * 100
print(jio[['Price', 'Profit', 'Margin_percentage']].sort_values(by='Margin_percentage',ascending=False).head())

    Price       Profit  Margin_percentage
13    249  1072.521739         430.731622
12    299  1018.678571         340.695174
9     119   337.842262         283.901060
8     199   393.385740         197.681276
6     259   386.508929         149.231247


#### 21. Complex Filter (Data > 1.5 & Streaming)

In [75]:
Streaming_filter = air['Benefits'].str.contains('Disney|Prime', case=False, na=False)

high_data = air['Data_Day'] >= 1.5

Q21 = air[Streaming_filter & high_data]

print(Q21[['Price','Data_Day', 'Benefits']].sort_values(by='Price',ascending=False))

    Price  Data_Day                    Benefits
0    3359       2.5     Disney + Hotstar 1 year
26   3359       2.5     Disney + Hotstar 1 year
2     999       2.5     Amazon prime membership
23    839       2.0   Disney + Hotstar 3 Months
3     839       2.0   Disney + Hotstar 3 Months
5     699       3.0     Amazon prime membership
7     599       3.0     Disney + Hotstar 1 year
25    599       3.0     Disney + Hotstar 1 year
9     499       2.0     Disney + Hotstar 1 year
24    499       2.0     Disney + Hotstar 1 year
12    399       2.5   Disney + Hotstar 3 Months
22    399       2.5   Disney + Hotstar 3 Months
13    359       2.0  Prime video mobile edition
